# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# load data from database
engine = create_engine('sqlite:///etl_pipeline.db')
df = pd.read_sql('select * from messages', engine)
df.head()


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
y = df.iloc[:, 4:39]


In [4]:
y.columns.tolist()

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather']

In [5]:
y['related'].replace(2, 1, inplace=True)

In [6]:
y['related'].unique()

array([1, 0])

### 2. Write a tokenization function to process your text data

In [7]:
import nltk
import re
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
def tokenize(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [8]:
for message in X[:5]:
    print(message)
    print(tokenize(message))

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti']
Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over']
Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name']
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '8090', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately']
says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred = pipeline.predict(X_test)


In [36]:
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

In [40]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
evaluation = {}
for column in y_test.columns:
    evaluation[column] = []
    evaluation[column].append(precision_score(y_test[column], y_pred_df[column]))
    evaluation[column].append(recall_score(y_test[column], y_pred_df[column]))
    evaluation[column].append(f1_score(y_test[column], y_pred_df[column]))


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [42]:
pd.DataFrame(evaluation)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather
0,0.826583,0.824348,0.0,0.740253,0.614035,0.714286,0.611111,0.0,0.400000,0.0,...,0.0,0.0,0.0,0.843805,0.868421,0.757895,1.000000,0.869792,0.833333,0.555556
1,0.924381,0.419841,0.0,0.521142,0.070850,0.100000,0.065476,0.0,0.040609,0.0,...,0.0,0.0,0.0,0.538719,0.251908,0.380282,0.028169,0.539580,0.070423,0.069252
2,0.872751,0.556338,0.0,0.611667,0.127042,0.175439,0.118280,0.0,0.073733,0.0,...,0.0,0.0,0.0,0.657599,0.390533,0.506448,0.054795,0.666002,0.129870,0.123153


In [22]:
from sklearn.metrics import classification_report
for i in range(35):
    print("Precision, Recall, F1 Score for {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Precision, Recall, F1 Score for related
             precision    recall  f1-score   support

          0       0.61      0.36      0.46      1523
          1       0.83      0.93      0.88      5031

avg / total       0.78      0.80      0.78      6554

Precision, Recall, F1 Score for request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5443
          1       0.81      0.36      0.50      1111

avg / total       0.87      0.88      0.86      6554

Precision, Recall, F1 Score for offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6529
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554

Precision, Recall, F1 Score for aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3859
          1       0.75      0.52      0.61      2695

avg / total       0.73      0.73

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
from sklearn.model_selection import GridSearchCV
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'tfidf__use_idf': (True, False)
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

In [24]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
print("Accuracy")
print((y_pred == y_test).mean())
for i in range(35):
    
    print("Precision, Recall, F1 Score for {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Accuracy
related                   0.791730
request                   0.882362
offer                     0.996186
aid_related               0.725664
medical_help              0.921575
medical_products          0.950717
search_and_rescue         0.973299
security                  0.978944
military                  0.964297
child_alone               1.000000
water                     0.946903
food                      0.924168
shelter                   0.935154
clothing                  0.986268
money                     0.974062
missing_people            0.987794
refugees                  0.965365
death                     0.960024
other_aid                 0.867562
infrastructure_related    0.938969
transport                 0.954989
buildings                 0.956973
electricity               0.980928
tools                     0.993287
hospitals                 0.991303
shops                     0.994812
aid_centers               0.987183
other_infrastructure      0.959567
weather_rel

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
# I am using AdaBoostClassifier instead of RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [15]:
# y_test['related']

In [17]:

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


In [18]:
from sklearn.metrics import classification_report
for i in range(35):
    print("Precision, Recall, F1 Score for {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Precision, Recall, F1 Score for related
             precision    recall  f1-score   support

          0       0.64      0.33      0.44      1524
          1       0.82      0.94      0.88      5030

avg / total       0.78      0.80      0.78      6554

Precision, Recall, F1 Score for request
             precision    recall  f1-score   support

          0       0.92      0.96      0.94      5459
          1       0.72      0.56      0.63      1095

avg / total       0.88      0.89      0.88      6554

Precision, Recall, F1 Score for offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.12      0.04      0.06        26

avg / total       0.99      1.00      0.99      6554

Precision, Recall, F1 Score for aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.81      3813
          1       0.76      0.61      0.68      2741

avg / total       0.76      0.76

### 9. Export your model as a pickle file

In [19]:
import pickle
filename = 'finalized_model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.